# 18 Singular vector transformation (SVT)
The novel SVD-based interface reduction approach is embedded in `pyFBS`. For more details on the singular vector transformation, refer to the documentation.

In [ ]:
import pyFBS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Numerical example
First, load a predefined datasets from the example directory. 

In [ ]:
pyFBS.download_lab_testbench()

In [ ]:
stl = r"./lab_testbench/STL/B.stl"
xlsx_pos = r"./lab_testbench/Measurements/ammeasurements.xlsx"

full_file = r"./lab_testbench/FEM/B.full"
ress_file = r"./lab_testbench/FEM/B.rst"

#### MK model
Prepare a MK model for the FRF synthetization. For more details check-out the *03_FRF_synthetization.ipynb* example. 

In [ ]:
MK = pyFBS.MK_model(ress_file,full_file,no_modes = 100,recalculate = False, scale=1000)

Add the STL and the corresponding mesh from FEM to the 3D view:

In [ ]:
view3D = pyFBS.view3D(show_origin= True)
mesh = view3D.add_stl(stl,name = "engine_mount",color = "#8FB1CC",opacity = .1)
view3D.plot.add_mesh(MK.mesh, scalars = np.zeros(MK.mesh.points.shape[0]),show_scalar_bar = False,name = "mesh",cmap = "coolwarm",show_edges = True);

To evaluate the FEM model the mode shapes can be animated: 

In [ ]:
select_mode = 6
_modeshape = MK.get_modeshape(select_mode)

mode_dict = pyFBS.utility.dict_animation(_modeshape,"modeshape",pts = MK.pts.copy(),mesh = MK.mesh)
view3D.add_modeshape(mode_dict,run_animation = True)

The mode shape deformation can also be cleared from the display:

In [ ]:
view3D.clear_modeshape()

Load the channel and datasets.

In [ ]:
df_imp = pd.read_excel(xlsx_pos, sheet_name='Impacts_B')
df_chn = pd.read_excel(xlsx_pos, sheet_name='Channels_B')

#view3D.show_imp(df_imp)
#view3D.show_chn(df_chn)

Currently only "snap-to-node" FRF synthetization is supported. Therefore, the locations of the channels and impacts have to updated to snap to the nearest node from FEM model.

In [ ]:
df_chn_up = MK.update_locations_df(df_chn)
df_imp_up = MK.update_locations_df(df_imp)

#view3D.show_chn(df_chn_up, color = "k",overwrite = False)
#view3D.show_imp(df_imp_up, color = "k",overwrite = False)

#### FRF synthetization
Finally, the FRFs synthetization at the channel/impact DoFs:

In [ ]:
MK.FRF_synth(df_chn_up,df_imp_up,f_start = 0,modal_damping = 0.003,frf_type = "accelerance")

## Singular vector transformation 
For the SVT to be applied, input ``df_imp_up`` and output channel ``df_chn_up`` informations must be selected. The grouping number ``grouping_no`` must be carefully assigned to the DoFs involved in the transformation process. This is pre-defined by the user when building the excel file for the design of experiments and successively imported in the channels (ref. channels) structure pandas.DataFrame under the label `` Grouping``. The SVT reduction matrices are then extracted by applying a truncated SVD on the chosen measured FRF datasets ``FRF``. Here, a number of 6 singular components (``no_svs``) is chosen along the whole frequency range of interest ``freq``. Take care of the requirements of the SVT when applied within a FBS decoupling application (see *FBS_decoupling_SVT.ipynb*). 

In [ ]:
k = 6
svt = pyFBS.SVT(df_chn_up,df_imp_up, freq = MK.freq, FRF = MK.FRF, grouping_no = [1,10],no_svs = k)


Show the CMIF plot and highlight the chosen singular values per frequency line

In [ ]:
plt.figure(figsize = (3,2))

plt.semilogy(svt.S[:,:k],color = "tab:blue")
plt.semilogy(svt.S[:,k:],"tab:blue",alpha = 0.2)
plt.xlim(0,2000)
plt.ylim(1e-5,1e4)

plt.xlabel("Frequency [Hz]");
plt.ylabel(r"CMIF $\Sigma$ [/]");

For the sake of the example, the transformation is applied here only on one subsystem ``FRF``. 

In [ ]:
_,_,FRF_sv= svt.apply_SVT(df_chn_up,df_imp_up, freq = MK.freq,FRF = MK.FRF)


Display the transformed FRFs. Note that since we have transformed the FRF datasets that was used for the extraction of the reduced singular bases, the resulting FRF matrix will be the corresponding matrix of singular values. In the following, the first 2 diagonal element of the matrix are plotted along the frequency axis.

In [ ]:
select_out = 0
select_in = 0

plt.figure(figsize = (8,6))
plt.semilogy(svt.freq,np.abs(FRF_sv[:,select_out,select_in]))
plt.semilogy(svt.freq,np.abs(FRF_sv[:,select_out+1,select_in+1]))

#### Consistency of the SVT

Check comparison of the measured FRFs with the SVT-filtered (reduced and back-transformed) ones.

In [ ]:
select_out = 2
select_in = 2

FRF_filt=np.zeros_like(svt.FRF,dtype = complex)
for i in np.arange(len(svt.freq)):
    FRF_filt[i,:,:]=svt.Fu[i,:,:] @ svt.FRF[i,:,:]@ svt.Ff[i,:,:]
        
plt.figure(figsize = (8,6))
plt.subplot(211)
plt.semilogy(svt.freq,np.abs(svt.FRF[:,select_out,select_in]),label = "measured")
plt.semilogy(svt.freq,np.abs(FRF_filt[:,select_out,select_in]),'--',label = "filtered")
plt.legend()

plt.subplot(413)
plt.plot(svt.freq,np.angle(svt.FRF[:,select_out,select_in]),label = "measured")
plt.plot(svt.freq,np.angle(FRF_filt[:,select_out,select_in]),'--',label = "filtered")